In [2]:
""" In this Colab, a MobileNet based model is created using Transfer Learning and then it is saved. It is loaded and trained on a training
dataset. A model can be saved in two formats, HDF5 format(Keras format) and SavedModel format(Tensorflow format). """

try:
  %tensorflow_version 2.x
except:
  pass

# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import logging
tf.get_logger().setLevel(logging.ERROR)
import time

TensorFlow 2.x selected.


In [0]:
# Preparing dataset
splits = tfds.Split.ALL.subsplit(weighted=(80,20))
(train_dataset,validation_dataset),info = tfds.load('cats_vs_dogs',split=splits,with_info=True,
                                                    as_supervised=True)

In [0]:
# Creating batches from dataset
def format_image(image,label):
    image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))/255.0
    return image,label
IMG_SIZE = 224
BATCH_SIZE = 32
num_examples = info.splits['train'].num_examples
train_batches = train_dataset.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_dataset.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [0]:
# Importing MobileNet model
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,input_shape=(IMG_SIZE,IMG_SIZE,3),trainable=False)

In [0]:
# Creating and Training the model
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(2,activation='softmax')
])

model.compile(loss = tf.losses.SparseCategoricalCrossentropy(),
          optimizer='adam',
          metrics=['accuracy'])

epochs = 3
history = model.fit(train_batches,
                   validation_data=validation_batches,
                   epochs=epochs)

Epoch 1/3
    582/Unknown - 60s 103ms/step - loss: 0.0564 - accuracy: 0.9804

In [0]:
# Testing the model performance
class_names = numpy.array(info.features['label'].names)
image_batch,label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()
predicted_batch = tf.squeeze(model.predict(image_batch)).numpy()
predicted_ids = np.argmax(predicted_batch,axis=-1)
predicted_class_names = class_names[predicted_ids]

plt.figure(figsize=(10,8))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplot_adjust(hspace=0.5)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(),color = color)
_ = plt.suptitle("Model Predictions")

In [0]:
# Saving the model in HDF5 format(.h5 extension)
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
model.save(export_path_keras)

In [0]:

reloaded = tf.keras.models.load_model(export_path_keras,
                                      custom_objects={'KerasLayer':hub.KerasLayer})
result_batch = model.predict(image_batch)
reloaded_batch = reloaded.predict(image_batch)

In [0]:
history = reloaded.fit(train_batches,
                      validation_data=validation_batches,
                      epochs=epochs)
